In [3]:
import uproot
import awkward
import numpy as np
from uproot_methods import TLorentzVectorArray

from coffea.processor import LazyDataFrame
from coffea.analysis_objects import JaggedCandidateArray

In [4]:
fn = '/hadoop/cms/store/user/mbryson/WH_hadronic/WH_had_750_1/nanoAOD/MBv93/WH_had_750_1__nanoAOD/merged/WH_had_750_1__nanoAOD_1.root'

file = uproot.open(fn)

In [5]:
tree = file['Events']
df = LazyDataFrame(tree, flatten=True)

jets = JaggedCandidateArray.candidatesfromcounts(
    df['nJet'],
    pt=df['Jet_pt'],
    eta=df['Jet_eta'],
    phi=df['Jet_phi'],
    mass=df['Jet_mass'],
    btag=df['Jet_btagDeepB'],
    #ctag=df['Jet_btagDeepC'],
)
muons = JaggedCandidateArray.candidatesfromcounts(
    df['nMuon'],
    pt=df['Muon_pt'],
    eta=df['Muon_eta'],
    phi=df['Muon_phi'],
    mass= 0 * df['Muon_pt'],
)

In [6]:
print(jets.pt)

[[545.5 500.0 96.75 ... 21.359375 21.140625 16.703125] [557.0 386.5 17.09375 15.296875] [181.625 169.125 120.5625 44.3125 29.765625 17.84375] ... [253.25 154.625 118.1875 ... 18.8125 15.4375 15.328125] [411.5 151.75 31.953125 ... 17.046875 15.8359375 15.265625] [548.0 284.0 44.21875]]


In [7]:
bjcut = ((jets.pt>30) & (abs(jets.eta)<2.4) & (jets.btag>0.4184))
bjets = jets[bjcut]
bjetpt = bjets.pt

print(bjetpt)

[[92.8125] [557.0] [] ... [154.625 81.75] [411.5 151.75] [284.0]]


In [8]:
bjetpt_sorted = bjetpt.argsort(ascending=False)
leadbjet = bjets[bjetpt_sorted==0]
subleadbjet = bjets[bjetpt_sorted==1]
leadbjets = bjets[bjetpt_sorted <= 1]
    
print(leadbjets.pt)

[[92.8125] [557.0] [] ... [154.625 81.75] [411.5 151.75] [284.0]]


In [9]:
dphi_leadbs_met = abs((leadbjets.phi - df['MET_phi'] + np.pi) % (2 * np.pi) - np.pi)
mt_b_met = np.sqrt(2*leadbjets.pt*df['MET_pt']*(1-np.cos(dphi_leadbs_met)))
        
print(mt_b_met)

[[416.0242] [744.61383] [] ... [580.8889 388.67062] [916.20306 521.55975] [495.392]]


In [10]:
sorted_min_mt_b_met = mt_b_met.argsort(ascending=True)

min_mt_b_met = mt_b_met[sorted_min_mt_b_met == 0]

print(min_mt_b_met)

[[416.0242] [744.61383] [] ... [388.67062] [521.55975] [495.392]]


In [14]:
mt_sel = (min_mt_b_met > 200).any()

In [15]:
print(min_mt_b_met[mt_sel])

[[416.0242] [744.61383] [505.925] ... [388.67062] [521.55975] [495.392]]


In [16]:
df['weight'][mt_sel]

array([4.932786e-08, 4.932786e-08, 4.932786e-08, ..., 4.932786e-08,
       4.932786e-08, 4.932786e-08], dtype=float32)